# Finding similar documents with Word2Vec and Soft Cosine Measure 

Soft Cosine Measure (SCM) [1, 4] is a promising new tool in machine learning that allows us to submit a query and return the most relevant documents. In **part 1**, we will show how you can compute SCM between two documents using the `inner_product` method. In **part 2**, we will use `SoftCosineSimilarity` to retrieve documents most similar to a query and compare the performance against other similarity measures.

First, however, we go through the basics of what Soft Cosine Measure is.

## Soft Cosine Measure basics

Soft Cosine Measure (SCM) is a method that allows us to assess the similarity between two documents in a meaningful way, even when they have no words in common. It uses a measure of similarity between words, which can be derived [2] using [word2vec][] [3] vector embeddings of words. It has been shown to outperform many of the state-of-the-art methods in the semantic text similarity task in the context of community question answering [2].

[word2vec]: https://radimrehurek.com/gensim/models/word2vec.html

SCM is illustrated below for two very similar sentences. The sentences have no words in common, but by modeling synonymy, SCM is able to accurately measure the similarity between the two sentences. The method also uses the bag-of-words vector representation of the documents (simply put, the word's frequencies in the documents). The intution behind the method is that we compute standard cosine similarity assuming that the document vectors are expressed in a non-orthogonal basis, where the angle between two basis vectors is derived from the angle between the word2vec embeddings of the corresponding words.

![Soft Cosine Measure](soft_cosine_tutorial.png)

This method was perhaps first introduced in the article “Soft Measure and Soft Cosine Measure: Measure of Features in Vector Space Model” by Grigori Sidorov, Alexander Gelbukh, Helena Gomez-Adorno, and David Pinto ([link to PDF](http://www.scielo.org.mx/pdf/cys/v18n3/v18n3a7.pdf)).

In this tutorial, we will learn how to use Gensim's SCM functionality, which consists of the `inner_product` method for one-off computation, and the `SoftCosineSimilarity` class for corpus-based similarity queries.

> **Note**:
>
> If you use this software, please consider citing [1] and [2].
>

## Running this notebook
You can download this [Jupyter notebook](http://jupyter.org/), and run it on your own computer, provided you have installed the `gensim`, `jupyter`, `sklearn`, `pyemd`, and `wmd` Python packages.

The notebook was run on an Ubuntu machine with an Intel core i7-6700HQ CPU 3.10GHz (4 cores) and 16 GB memory. Assuming all resources required by the notebook have already been downloaded, running the entire notebook on this machine takes about 30 minutes.

In [1]:
# Initialize logging.
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Part 1: Computing the Soft Cosine Measure

To use SCM, we need some word embeddings first of all. You could train a [word2vec][] (see tutorial [here](http://rare-technologies.com/word2vec-tutorial/)) model on some corpus, but we will use pre-trained word2vec embeddings.

[word2vec]: https://radimrehurek.com/gensim/models/word2vec.html

Let's create some sentences to compare.

In [2]:
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()
sentence_orange = 'Having a tough time finding an orange juice press machine?'.lower().split()

The first two sentences have very similar content, and as such the SCM should be large. Before we compute the SCM, we want to remove stopwords ("the", "to", etc.), as these do not contribute a lot to the information in the sentences.

In [3]:
# Import and download stopwords from NLTK.
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.

# Remove stopwords.
stop_words = stopwords.words('english')
sentence_obama = [w for w in sentence_obama if w not in stop_words]
sentence_president = [w for w in sentence_president if w not in stop_words]
sentence_orange = [w for w in sentence_orange if w not in stop_words]

# Prepare a dictionary and a corpus.
from gensim import corpora
documents = [sentence_obama, sentence_president, sentence_orange]
dictionary = corpora.Dictionary(documents)

# Convert the sentences into bag-of-words vectors.
sentence_obama = dictionary.doc2bow(sentence_obama)
sentence_president = dictionary.doc2bow(sentence_president)
sentence_orange = dictionary.doc2bow(sentence_orange)

[nltk_data] Downloading package stopwords to /home/misha/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
2019-06-17 10:46:35,031 : INFO : 'pattern' package not found; tag filters are not available for English
2019-06-17 10:46:35,036 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 10:46:35,037 : INFO : built Dictionary(14 unique tokens: ['illinois', 'media', 'obama', 'speaks', 'chicago']...) from 3 documents (total 15 corpus positions)


Now, as we mentioned earlier, we will be using some downloaded pre-trained embeddings. Note that the embeddings we have chosen here require a lot of memory. We will use the embeddings to construct a term similarity matrix that will be used by the `inner_product` method.

In [4]:
%%time
import gensim.downloader as api
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix

w2v_model = api.load("glove-wiki-gigaword-50")
similarity_index = WordEmbeddingSimilarityIndex(w2v_model)
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary)

2019-06-17 10:46:35,428 : INFO : loading projection weights from /home/misha/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz
2019-06-17 10:46:35,429 : WARNING : this function is deprecated, use smart_open.open instead
2019-06-17 10:47:07,511 : INFO : loaded (400000, 50) matrix from /home/misha/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz
2019-06-17 10:47:07,512 : INFO : constructing a sparse term similarity matrix using <gensim.models.keyedvectors.WordEmbeddingSimilarityIndex object at 0x7fa0c2347d68>
2019-06-17 10:47:07,512 : INFO : iterating over columns in dictionary order
2019-06-17 10:47:07,514 : INFO : PROGRESS: at 7.14% columns (1 / 14, 7.142857% density, 7.142857% projected density)
2019-06-17 10:47:07,516 : INFO : precomputing L2-norms of word weight vectors
2019-06-17 10:47:08,055 : INFO : constructed a sparse term similarity matrix with 11.224490% density


CPU times: user 30.4 s, sys: 1.27 s, total: 31.7 s
Wall time: 33 s


Let's compute SCM using the `inner_product` method.

In [5]:
similarity = similarity_matrix.inner_product(sentence_obama, sentence_president, normalized=True)
print('similarity = %.4f' % similarity)

similarity = 0.3790


Let's try the same thing with two completely unrelated sentences. Notice that the similarity is smaller.

In [6]:
similarity = similarity_matrix.inner_product(sentence_obama, sentence_orange, normalized=True)
print('similarity = %.4f' % similarity)

similarity = 0.1108


## Part 2: Similarity queries using `SoftCosineSimilarity`
You can use SCM to get the most similar documents to a query, using the `SoftCosineSimilarity` class. Its interface is similar to what is described in the [Similarity Queries](https://radimrehurek.com/gensim/tut3.html) Gensim tutorial.

### Qatar Living unannotated dataset
Contestants solving the community question answering task in the [SemEval 2016][semeval16] and [2017][semeval17] competitions had an unannotated dataset of 189,941 questions and 1,894,456 comments from the [Qatar Living][ql] discussion forums. As our first step, we will use the same dataset to build a corpus.

[semeval16]: http://alt.qcri.org/semeval2016/task3/
[semeval17]: http://alt.qcri.org/semeval2017/task3/
[ql]: http://www.qatarliving.com/forum

In [7]:
%%time
from itertools import chain
import json
from re import sub
from os.path import isfile

import gensim.downloader as api
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk import download


download("stopwords")  # Download stopwords list.
stopwords = set(stopwords.words("english"))

def preprocess(doc):
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

corpus = list(chain(*[
    chain(
        [preprocess(thread["RelQuestion"]["RelQSubject"]), preprocess(thread["RelQuestion"]["RelQBody"])],
        [preprocess(relcomment["RelCText"]) for relcomment in thread["RelComments"]])
    for thread in api.load("semeval-2016-2017-task3-subtaskA-unannotated")]))

print("Number of documents: %d" % len(documents))

[nltk_data] Downloading package stopwords to /home/misha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2019-06-17 10:47:08,404 : WARNING : this function is deprecated, use smart_open.open instead


Number of documents: 3
CPU times: user 2min 46s, sys: 2.34 s, total: 2min 48s
Wall time: 2min 52s


Using the corpus we have just build, we will now construct a [dictionary][], a [TF-IDF model][tfidf], a [word2vec model][word2vec], and a term similarity matrix.

[dictionary]: https://radimrehurek.com/gensim/corpora/dictionary.html
[tfidf]: https://radimrehurek.com/gensim/models/tfidfmodel.html
[word2vec]: https://radimrehurek.com/gensim/models/word2vec.html

In [8]:
%%time
from multiprocessing import cpu_count

from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import Word2Vec
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix

dictionary = Dictionary(corpus)
tfidf = TfidfModel(dictionary=dictionary)
w2v_model = Word2Vec(corpus, workers=cpu_count(), min_count=5, size=300, seed=12345)
similarity_index = WordEmbeddingSimilarityIndex(w2v_model.wv)
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf, nonzero_limit=100)

2019-06-17 10:50:00,986 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 10:50:01,293 : INFO : adding document #10000 to Dictionary(20088 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:01,574 : INFO : adding document #20000 to Dictionary(29692 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:01,932 : INFO : adding document #30000 to Dictionary(37971 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:02,293 : INFO : adding document #40000 to Dictionary(43930 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:02,551 : INFO : adding document #50000 to Dictionary(49340 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:02,817 : INFO : adding document #60000 to Dictionary(54734 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:03

2019-06-17 10:50:21,125 : INFO : adding document #550000 to Dictionary(195951 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:21,644 : INFO : adding document #560000 to Dictionary(197956 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:22,189 : INFO : adding document #570000 to Dictionary(200145 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:22,809 : INFO : adding document #580000 to Dictionary(201859 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:23,466 : INFO : adding document #590000 to Dictionary(203724 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:24,117 : INFO : adding document #600000 to Dictionary(205607 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:24,644 : INFO : adding document #610000 to Dictionary(207387 unique tokens: [

2019-06-17 10:50:52,177 : INFO : adding document #1100000 to Dictionary(293838 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:52,927 : INFO : adding document #1110000 to Dictionary(295273 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:53,529 : INFO : adding document #1120000 to Dictionary(296816 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:54,091 : INFO : adding document #1130000 to Dictionary(298552 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:54,500 : INFO : adding document #1140000 to Dictionary(299628 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:55,158 : INFO : adding document #1150000 to Dictionary(301139 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:50:55,844 : INFO : adding document #1160000 to Dictionary(302566 unique to

2019-06-17 10:51:09,616 : INFO : adding document #1640000 to Dictionary(374282 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:51:09,885 : INFO : adding document #1650000 to Dictionary(375746 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:51:10,223 : INFO : adding document #1660000 to Dictionary(377073 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:51:10,535 : INFO : adding document #1670000 to Dictionary(378393 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:51:10,805 : INFO : adding document #1680000 to Dictionary(379812 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:51:11,088 : INFO : adding document #1690000 to Dictionary(380895 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:51:11,452 : INFO : adding document #1700000 to Dictionary(384739 unique to

2019-06-17 10:51:24,664 : INFO : adding document #2180000 to Dictionary(451840 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:51:24,940 : INFO : adding document #2190000 to Dictionary(453020 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:51:25,245 : INFO : adding document #2200000 to Dictionary(454160 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:51:25,496 : INFO : adding document #2210000 to Dictionary(455302 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:51:25,781 : INFO : adding document #2220000 to Dictionary(456657 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:51:26,053 : INFO : adding document #2230000 to Dictionary(457752 unique tokens: ['blocks', 'cnn', 'facebook', 'minsitry', 'thailand']...)
2019-06-17 10:51:26,316 : INFO : adding document #2240000 to Dictionary(458938 unique to

...

2019-06-17 10:59:14,945 : INFO : PROGRESS: at 17.50% columns (81001 / 462807, 0.000243% density, 0.000372% projected density)
2019-06-17 10:59:15,083 : INFO : PROGRESS: at 17.72% columns (82001 / 462807, 0.000244% density, 0.000371% projected density)
2019-06-17 10:59:15,156 : INFO : PROGRESS: at 17.93% columns (83001 / 462807, 0.000244% density, 0.000370% projected density)
2019-06-17 10:59:15,193 : INFO : PROGRESS: at 18.15% columns (84001 / 462807, 0.000244% density, 0.000368% projected density)
2019-06-17 10:59:15,392 : INFO : PROGRESS: at 18.37% columns (85001 / 462807, 0.000244% density, 0.000369% projected density)
2019-06-17 10:59:15,548 : INFO : PROGRESS: at 18.58% columns (86001 / 462807, 0.000245% density, 0.000370% projected density)
2019-06-17 10:59:15,651 : INFO : PROGRESS: at 18.80% columns (87001 / 462807, 0.000245% density, 0.000369% projected density)
2019-06-17 10:59:15,730 : INFO : PROGRESS: at 19.01% columns (88001 / 462807, 0.000245% density, 0.000367% projected d

2019-06-17 10:59:27,252 : INFO : PROGRESS: at 31.55% columns (146001 / 462807, 0.000263% density, 0.000363% projected density)
2019-06-17 10:59:27,328 : INFO : PROGRESS: at 31.76% columns (147001 / 462807, 0.000263% density, 0.000363% projected density)
2019-06-17 10:59:27,399 : INFO : PROGRESS: at 31.98% columns (148001 / 462807, 0.000263% density, 0.000363% projected density)
2019-06-17 10:59:27,498 : INFO : PROGRESS: at 32.20% columns (149001 / 462807, 0.000263% density, 0.000363% projected density)
2019-06-17 10:59:27,727 : INFO : PROGRESS: at 32.41% columns (150001 / 462807, 0.000264% density, 0.000364% projected density)
2019-06-17 10:59:27,858 : INFO : PROGRESS: at 32.63% columns (151001 / 462807, 0.000264% density, 0.000364% projected density)
2019-06-17 10:59:27,938 : INFO : PROGRESS: at 32.84% columns (152001 / 462807, 0.000264% density, 0.000363% projected density)
2019-06-17 10:59:28,166 : INFO : PROGRESS: at 33.06% columns (153001 / 462807, 0.000265% density, 0.000364% pro

2019-06-17 10:59:36,331 : INFO : PROGRESS: at 45.59% columns (211001 / 462807, 0.000284% density, 0.000364% projected density)
2019-06-17 10:59:36,391 : INFO : PROGRESS: at 45.81% columns (212001 / 462807, 0.000284% density, 0.000364% projected density)
2019-06-17 10:59:36,453 : INFO : PROGRESS: at 46.02% columns (213001 / 462807, 0.000284% density, 0.000363% projected density)
2019-06-17 10:59:36,556 : INFO : PROGRESS: at 46.24% columns (214001 / 462807, 0.000284% density, 0.000363% projected density)
2019-06-17 10:59:36,711 : INFO : PROGRESS: at 46.46% columns (215001 / 462807, 0.000284% density, 0.000363% projected density)
2019-06-17 10:59:36,781 : INFO : PROGRESS: at 46.67% columns (216001 / 462807, 0.000284% density, 0.000363% projected density)
2019-06-17 10:59:36,945 : INFO : PROGRESS: at 46.89% columns (217001 / 462807, 0.000285% density, 0.000363% projected density)
2019-06-17 10:59:36,997 : INFO : PROGRESS: at 47.10% columns (218001 / 462807, 0.000285% density, 0.000363% pro

2019-06-17 10:59:59,075 : INFO : PROGRESS: at 59.64% columns (276001 / 462807, 0.000321% density, 0.000393% projected density)
2019-06-17 10:59:59,593 : INFO : PROGRESS: at 59.85% columns (277001 / 462807, 0.000322% density, 0.000393% projected density)
2019-06-17 11:00:00,401 : INFO : PROGRESS: at 60.07% columns (278001 / 462807, 0.000323% density, 0.000395% projected density)
2019-06-17 11:00:01,359 : INFO : PROGRESS: at 60.28% columns (279001 / 462807, 0.000325% density, 0.000396% projected density)
2019-06-17 11:00:02,584 : INFO : PROGRESS: at 60.50% columns (280001 / 462807, 0.000327% density, 0.000399% projected density)


TypeError: cannot unpack non-iterable numpy.float32 object

### Evaluation
Next, we will load the validation and test datasets that were used by the SemEval 2016 and 2017 contestants. The datasets contain 208 original questions posted by the forum members. For each question, there is a list of 10 threads with a human annotation denoting whether or not the thread is relevant to the original question. Our task will be to order the threads so that relevant threads rank above irrelevant threads.

In [9]:
datasets = api.load("semeval-2016-2017-task3-subtaskBC")

2019-06-17 11:00:04,683 : WARNING : this function is deprecated, use smart_open.open instead


Finally, we will perform an evaluation to compare three unsupervised similarity measures – the Soft Cosine Measure, two different implementations of the [Word Mover's Distance][wmd], and standard cosine similarity. We will use the [Mean Average Precision (MAP)][map] as an evaluation measure and 10-fold cross-validation to get an estimate of the variance of MAP for each similarity measure.

[wmd]: http://vene.ro/blog/word-movers-distance-in-python.html
[map]: https://medium.com/@pds.bangalore/mean-average-precision-abd77d0b9a7e

In [10]:
!pip install wmd

You are using pip version 19.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
from math import isnan
from time import time

from gensim.similarities import MatrixSimilarity, WmdSimilarity, SoftCosineSimilarity
import numpy as np
from sklearn.model_selection import KFold
from wmd import WMD

def produce_test_data(dataset):
    for orgquestion in datasets[dataset]:
        query = preprocess(orgquestion["OrgQSubject"]) + preprocess(orgquestion["OrgQBody"])
        documents = [
            preprocess(thread["RelQuestion"]["RelQSubject"]) + preprocess(thread["RelQuestion"]["RelQBody"])
            for thread in orgquestion["Threads"]]
        relevance = [
            thread["RelQuestion"]["RELQ_RELEVANCE2ORGQ"] in ("PerfectMatch", "Relevant")
            for thread in orgquestion["Threads"]]
        yield query, documents, relevance

def cossim(query, documents):
    # Compute cosine similarity between the query and the documents.
    query = tfidf[dictionary.doc2bow(query)]
    index = MatrixSimilarity(
        tfidf[[dictionary.doc2bow(document) for document in documents]],
        num_features=len(dictionary))
    similarities = index[query]
    return similarities

def softcossim(query, documents):
    # Compute Soft Cosine Measure between the query and the documents.
    query = tfidf[dictionary.doc2bow(query)]
    index = SoftCosineSimilarity(
        tfidf[[dictionary.doc2bow(document) for document in documents]],
        similarity_matrix)
    similarities = index[query]
    return similarities

def wmd_gensim(query, documents):
    # Compute Word Mover's Distance as implemented in PyEMD by William Mayner
    # between the query and the documents.
    index = WmdSimilarity(documents, w2v_model)
    similarities = index[query]
    return similarities

def wmd_relax(query, documents):
    # Compute Word Mover's Distance as implemented in WMD by Source{d}
    # between the query and the documents.
    words = [word for word in set(chain(query, *documents)) if word in w2v_model.wv]
    indices, words = zip(*sorted((
        (index, word) for (index, _), word in zip(dictionary.doc2bow(words), words))))
    query = dict(tfidf[dictionary.doc2bow(query)])
    query = [
        (new_index, query[dict_index])
        for new_index, dict_index in enumerate(indices)
        if dict_index in query]
    documents = [dict(tfidf[dictionary.doc2bow(document)]) for document in documents]
    documents = [[
        (new_index, document[dict_index])
        for new_index, dict_index in enumerate(indices)
        if dict_index in document] for document in documents]
    embeddings = np.array([w2v_model.wv[word] for word in words], dtype=np.float32)
    nbow = dict(((index, list(chain([None], zip(*document)))) for index, document in enumerate(documents)))
    nbow["query"] = tuple([None] + list(zip(*query)))
    distances = WMD(embeddings, nbow, vocabulary_min=1).nearest_neighbors("query")
    similarities = [-distance for _, distance in sorted(distances)]
    return similarities

strategies = {
    "cossim" : cossim,
    "softcossim": softcossim,
    "wmd-gensim": wmd_gensim,
    "wmd-relax": wmd_relax}

def evaluate(split, strategy):
    # Perform a single round of evaluation.
    results = []
    start_time = time()
    for query, documents, relevance in split:
        similarities = strategies[strategy](query, documents)
        assert len(similarities) == len(documents)
        precision = [
            (num_correct + 1) / (num_total + 1) for num_correct, num_total in enumerate(
                num_total for num_total, (_, relevant) in enumerate(
                    sorted(zip(similarities, relevance), reverse=True)) if relevant)]
        average_precision = np.mean(precision) if precision else 0.0
        results.append(average_precision)
    return (np.mean(results) * 100, time() - start_time)

def crossvalidate(args):
    # Perform a cross-validation.
    dataset, strategy = args
    test_data = np.array(list(produce_test_data(dataset)))
    kf = KFold(n_splits=10)
    samples = []
    for _, test_index in kf.split(test_data):
        samples.append(evaluate(test_data[test_index], strategy))
    return (np.mean(samples, axis=0), np.std(samples, axis=0))

In [ ]:
%%time
from multiprocessing import Pool

args_list = [
    (dataset, technique)
    for dataset in ("2016-test", "2017-test")
    for technique in ("softcossim", "wmd-gensim", "wmd-relax", "cossim")]
with Pool() as pool:
    results = pool.map(crossvalidate, args_list)

2019-06-17 11:00:13,689 : INFO : creating matrix with 10 documents and 462807 features
2019-06-17 11:00:13,822 : INFO : precomputing L2-norms of word weight vectors
2019-06-17 11:00:14,044 : INFO : creating matrix with 10 documents and 462807 features
2019-06-17 11:00:14,270 : INFO : precomputing L2-norms of word weight vectors
2019-06-17 11:00:14,795 : INFO : creating matrix with 10 documents and 462807 features
2019-06-17 11:00:14,757 : INFO : Vocabulary size: 35 500
2019-06-17 11:00:14,907 : INFO : WCD
2019-06-17 11:00:14,927 : INFO : 0.0
2019-06-17 11:00:14,919 : INFO : creating matrix with 10 documents and 462807 features
2019-06-17 11:00:14,950 : INFO : First K WMD
2019-06-17 11:00:15,013 : INFO : creating matrix with 10 documents and 462807 features
2019-06-17 11:00:15,186 : INFO : Vocabulary size: 19 500
2019-06-17 11:00:15,209 : INFO : WCD
2019-06-17 11:00:15,184 : INFO : [(-19.50642204284668, 6), (-18.93822479248047, 4), (-19.099836349487305, 7), (-18.120803833007812, 5), (-1

2019-06-17 11:00:16,974 : INFO : First K WMD
2019-06-17 11:00:16,997 : INFO : [(-18.612998962402344, 7), (-17.614749908447266, 0), (-17.181663513183594, 6), (-16.649444580078125, 8), (-16.556007385253906, 9), (-12.592411041259766, 3), (-16.570131301879883, 5), (-11.052681922912598, 2), (-16.47679328918457, 4), (-16.522056579589844, 1)]
2019-06-17 11:00:17,027 : INFO : 0.0
2019-06-17 11:00:17,045 : INFO : P&P
2019-06-17 11:00:17,053 : INFO : stopped by early_stop condition
2019-06-17 11:00:17,093 : INFO : creating matrix with 10 documents and 462807 features
2019-06-17 11:00:17,121 : INFO : Vocabulary size: 6 500
2019-06-17 11:00:17,113 : INFO : creating matrix with 10 documents and 462807 features
2019-06-17 11:00:17,136 : INFO : WCD
2019-06-17 11:00:17,131 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:00:17,158 : INFO : Vocabulary size: 24 500
2019-06-17 11:00:17,170 : INFO : built Dictionary(35 unique tokens: ['abu', 'airport', 'apply', 'architect', 'ar

2019-06-17 11:00:18,065 : INFO : 0.1
2019-06-17 11:00:18,102 : INFO : Vocabulary size: 16 500
2019-06-17 11:00:18,111 : INFO : creating matrix with 10 documents and 462807 features
2019-06-17 11:00:18,147 : INFO : P&P
2019-06-17 11:00:18,150 : INFO : stopped by early_stop condition
2019-06-17 11:00:18,143 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:00:18,169 : INFO : built Dictionary(63 unique tokens: ['accepted', 'advice', 'anyone', 'appreciated', 'april']...) from 2 documents (total 77 corpus positions)
2019-06-17 11:00:18,174 : INFO : WCD
2019-06-17 11:00:18,182 : INFO : 0.0
2019-06-17 11:00:18,221 : INFO : creating matrix with 10 documents and 462807 features
2019-06-17 11:00:18,204 : INFO : First K WMD
2019-06-17 11:00:18,268 : INFO : [(-20.066619873046875, 4), (-19.894027709960938, 5), (-19.66065788269043, 2), (-19.0922908782959, 1), (-19.50944709777832, 7), (-18.448623657226562, 3), (-14.045564651489258, 0), (-18.474266052246094, 8), (-18.9484672

2019-06-17 11:00:19,388 : INFO : built Dictionary(34 unique tokens: ['applied', 'applying', 'ask', 'certificate', 'change']...) from 2 documents (total 56 corpus positions)
2019-06-17 11:00:19,405 : INFO : [(-22.74655532836914, 8), (-22.712932586669922, 9), (-22.07628059387207, 5), (-21.98308563232422, 2), (-19.768983840942383, 4), (-21.54038429260254, 6), (-21.95960235595703, 0), (-18.395387649536133, 3), (-21.559354782104492, 7), (-19.146608352661133, 1)]
2019-06-17 11:00:19,390 : INFO : creating matrix with 10 documents and 462807 features
2019-06-17 11:00:19,407 : INFO : 0.0
2019-06-17 11:00:19,427 : INFO : Vocabulary size: 27 500
2019-06-17 11:00:19,432 : INFO : P&P
2019-06-17 11:00:19,439 : INFO : stopped by early_stop condition
2019-06-17 11:00:19,440 : INFO : WCD
2019-06-17 11:00:19,457 : INFO : 0.0
2019-06-17 11:00:19,462 : INFO : First K WMD
2019-06-17 11:00:19,454 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:00:19,473 : INFO : built Dictionary

2019-06-17 11:00:20,305 : INFO : WCD
2019-06-17 11:00:20,311 : INFO : creating matrix with 10 documents and 462807 features
2019-06-17 11:00:20,310 : INFO : creating matrix with 10 documents and 462807 features
2019-06-17 11:00:20,306 : INFO : P&P
2019-06-17 11:00:20,319 : INFO : 0.0
2019-06-17 11:00:20,334 : INFO : First K WMD
2019-06-17 11:00:20,327 : INFO : built Dictionary(36 unique tokens: ['around', 'closed', 'expired', 'garvey', 'got']...) from 2 documents (total 43 corpus positions)
2019-06-17 11:00:20,330 : INFO : stopped by early_stop condition
2019-06-17 11:00:20,361 : INFO : [(-19.928001403808594, 4), (-19.701377868652344, 9), (-19.740556716918945, 5), (-19.489063262939453, 8), (-14.717412948608398, 3), (-19.454524993896484, 1), (-18.943920135498047, 7), (-18.111894607543945, 6), (-17.66033935546875, 2), (-0.0, 0)]
2019-06-17 11:00:20,369 : INFO : 0.0
2019-06-17 11:00:20,379 : INFO : P&P
2019-06-17 11:00:20,374 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


...

2019-06-17 11:01:23,637 : INFO : built Dictionary(47 unique tokens: ['admissions', 'admit', 'advance', 'back', 'birla']...) from 2 documents (total 61 corpus positions)
2019-06-17 11:01:23,695 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:23,733 : INFO : built Dictionary(27 unique tokens: ['companies', 'currently', 'daycare', 'employers', 'facility']...) from 2 documents (total 34 corpus positions)
2019-06-17 11:01:23,761 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:23,787 : INFO : built Dictionary(47 unique tokens: ['advise', 'airways', 'anybody', 'british', 'children']...) from 2 documents (total 61 corpus positions)
2019-06-17 11:01:23,821 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:23,840 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:23,847 : INFO : built Dictionary(48 unique tokens: ['administration', 'agent', 'body', 'business', 'change']...) 

2019-06-17 11:01:25,293 : INFO : Removed 2 and 0 OOV words from document 1 and 2 (respectively).
2019-06-17 11:01:25,308 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:25,325 : INFO : built Dictionary(31 unique tokens: ['american', 'anyone', 'chiropractor', 'clinic', 'closed']...) from 2 documents (total 33 corpus positions)
2019-06-17 11:01:25,363 : INFO : Removed 1 and 0 OOV words from document 1 and 2 (respectively).
2019-06-17 11:01:25,384 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:25,401 : INFO : built Dictionary(38 unique tokens: ['advance', 'baby', 'bcg', 'better', 'child']...) from 2 documents (total 45 corpus positions)
2019-06-17 11:01:25,438 : INFO : precomputing L2-norms of word weight vectors
2019-06-17 11:01:25,632 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:25,650 : INFO : built Dictionary(17 unique tokens: ['anyone', 'car', 'company', 'departure', 'drive']...) from 

2019-06-17 11:01:27,354 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:27,388 : INFO : built Dictionary(51 unique tokens: ['anyone', 'apart', 'bring', 'customs', 'expect']...) from 2 documents (total 61 corpus positions)
2019-06-17 11:01:27,396 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:27,425 : INFO : built Dictionary(30 unique tokens: ['appreciate', 'bringing', 'family', 'help', 'limitations']...) from 2 documents (total 43 corpus positions)
2019-06-17 11:01:27,469 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:27,490 : INFO : built Dictionary(39 unique tokens: ['anyone', 'apply', 'completed', 'country', 'exit']...) from 2 documents (total 60 corpus positions)
2019-06-17 11:01:27,500 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:27,502 : INFO : built Dictionary(48 unique tokens: ['allowed', 'come', 'doha', 'dresses', 'formals']...) from 2 documents 

2019-06-17 11:01:29,304 : INFO : built Dictionary(18 unique tokens: ['home', 'room', 'spend', 'time', 'arrange']...) from 2 documents (total 25 corpus positions)
2019-06-17 11:01:29,329 : INFO : built Dictionary(44 unique tokens: ['abaya', 'arab', 'curious', 'dying', 'hey']...) from 2 documents (total 56 corpus positions)
2019-06-17 11:01:29,323 : INFO : Removed 1 and 1 OOV words from document 1 and 2 (respectively).
2019-06-17 11:01:29,352 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:29,356 : INFO : built Dictionary(20 unique tokens: ['compound', 'gardens', 'information', 'someone', 'tell']...) from 2 documents (total 23 corpus positions)
2019-06-17 11:01:29,377 : INFO : Removed 1 and 1 OOV words from document 1 and 2 (respectively).
2019-06-17 11:01:29,384 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:29,380 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:29,394 : INFO : built Diction

2019-06-17 11:01:30,914 : INFO : Removed 1 and 0 OOV words from document 1 and 2 (respectively).
2019-06-17 11:01:30,934 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:30,953 : INFO : built Dictionary(49 unique tokens: ['account', 'across', 'aka', 'axis', 'bank']...) from 2 documents (total 61 corpus positions)
2019-06-17 11:01:31,029 : INFO : Removed 1 and 0 OOV words from document 1 and 2 (respectively).
2019-06-17 11:01:31,061 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:31,076 : INFO : built Dictionary(49 unique tokens: ['account', 'appeared', 'bank', 'bussiness', 'call']...) from 2 documents (total 58 corpus positions)
2019-06-17 11:01:31,136 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-06-17 11:01:31,165 : INFO : built Dictionary(32 unique tokens: ['accomodation', 'day', 'doha', 'free', 'give']...) from 2 documents (total 37 corpus positions)
2019-06-17 11:01:31,196 : INFO : adding document

The table below shows the pointwise estimates of means and standard variances for MAP scores and elapsed times. Baselines and winners for each year are displayed in bold. We can see that the Soft Cosine Measure gives a strong performance on both the 2016 and the 2017 dataset.

In [ ]:
from IPython.display import display, Markdown

output = []
baselines = [
    (("2016-test", "**Winner (UH-PRHLT-primary)**"), ((76.70, 0), (0, 0))),
    (("2016-test", "**Baseline 1 (IR)**"), ((74.75, 0), (0, 0))),
    (("2016-test", "**Baseline 2 (random)**"), ((46.98, 0), (0, 0))),
    (("2017-test", "**Winner (SimBow-primary)**"), ((47.22, 0), (0, 0))),
    (("2017-test", "**Baseline 1 (IR)**"), ((41.85, 0), (0, 0))),
    (("2017-test", "**Baseline 2 (random)**"), ((29.81, 0), (0, 0)))]
table_header = ["Dataset | Strategy | MAP score | Elapsed time (sec)", ":---|:---|:---|---:"]
for row, ((dataset, technique), ((mean_map_score, mean_duration), (std_map_score, std_duration))) \
        in enumerate(sorted(chain(zip(args_list, results), baselines), key=lambda x: (x[0][0], -x[1][0][0]))):
    if row % (len(strategies) + 3) == 0:
        output.extend(chain(["\n"], table_header))
    map_score = "%.02f ±%.02f" % (mean_map_score, std_map_score)
    duration = "%.02f ±%.02f" % (mean_duration, std_duration) if mean_duration else ""
    output.append("%s|%s|%s|%s" % (dataset, technique, map_score, duration))

display(Markdown('\n'.join(output)))

## References

1. Grigori Sidorov et al. *Soft Similarity and Soft Cosine Measure: Similarity of Features in Vector Space Model*, 2014. ([link to PDF](http://www.scielo.org.mx/pdf/cys/v18n3/v18n3a7.pdf))
2. Delphine Charlet and Geraldine Damnati, SimBow at SemEval-2017 Task 3: Soft-Cosine Semantic Similarity between Questions for Community Question Answering, 2017. ([link to PDF](http://www.aclweb.org/anthology/S17-2051))
3. Thomas Mikolov et al. Efficient Estimation of Word Representations in Vector Space, 2013. ([link to PDF](https://arxiv.org/pdf/1301.3781.pdf))
4. Vít Novotný. *Implementation Notes for the Soft Cosine Measure*, 2018. ([link to PDF](https://arxiv.org/pdf/1808.09407))